In [ ]:
import os  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path  # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8



In [ ]:
# Set Variables
site_list = ['Indeed', 'Glassdoor', 'LinkedIn']
nan_list = [None, 'None', '', ' ', [], -1, '-1', 0, '0', 'nan', np.nan, 'Nan']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
alpha = 0.050
normality_tests_labels = ['Statistic', 'p-value']
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']
ivs = ['Gender', 'Age']
ivs_all = [
    'Gender',
    'Gender_Num',
    'Gender_Female',
    'Gender_Mixed',
    'Gender_Male',
    'Gender_Female_n',
    'Gender_Male_n',
    'Gender_Female_% per Sector',
    'Gender_Male_% per Sector',
    'Age',
    'Age_Num',
    'Age_Older',
    'Age_Mixed',
    'Age_Younger',
    'Age_Older_n',
    'Age_Younger_n',
    'Age_Older_% per Sector',
    'Age_Younger_% per Sector',
]
ivs_count = [
    'Gender_Female_n',
    'Gender_Male_n',
    'Age_Older_n',
    'Age_Younger_n',
]
ivs_gender_count = [
    'Gender_Female_n',
    'Gender_Male_n',
]
ivs_age_count = [
    'Age_Older_n',
    'Age_Younger_n',
]
ivs_perc = [
    'Gender_Female_% per Sector',
    'Gender_Male_% per Sector',
    'Age_Older_% per Sector',
    'Age_Younger_% per Sector',
]
ivs_gender_perc = [
    'Gender_Female_% per Sector',
    'Gender_Male_% per Sector',
]
ivs_age_perc = [
    'Age_Older_% per Sector',
    'Age_Younger_% per Sector',
]
ivs_all_dummy_num = [
    'Gender_Num',
    'Gender_Female',
    'Gender_Mixed',
    'Gender_Male',
    'Age_Num',
    'Age_Older',
    'Age_Mixed',
    'Age_Younger',
]
ivs_all_dummy = [
    'Gender_Female',
    'Gender_Mixed',
    'Gender_Male',
    'Age_Older',
    'Age_Mixed',
    'Age_Younger',
]
ivs_gender_dummy_num = [
    'Gender_Num',
    'Gender_Female',
    'Gender_Mixed',
    'Gender_Male',
]
ivs_gender_dummy = [
    'Gender_Female',
    'Gender_Mixed',
    'Gender_Male',
]
ivs_age_dummy_num = [
    'Age_Num',
    'Age_Older',
    'Age_Mixed',
    'Age_Younger',
]
ivs_age_dummy_num = [
    'Age_Older',
    'Age_Mixed',
    'Age_Younger',
]
gender_order = ['Female', 'Mixed Gender', 'Male']
age_order = ['Older', 'Mixed Age', 'Younger']
platform_order = ['LinkedIn', 'Indeed', 'Glassdoor']
ivs_dict = {'Gender': gender_order, 'Age': age_order}
dvs = [
    'Warmth', 'Competence'
]
dvs_all = [
    'Warmth', 'Competence', 'Warmth_Probability', 'Competence_Probability'
]
dvs_prob = [
    'Warmth_Probability', 'Competence_Probability'
]

cat_list = [
    'Job ID',
    'Gender',
    'Gender_Female',
    'Gender_Male',
    'Gender_Mixed',
    'Age',
    'Age_Older',
    'Age_Younger',
    'Gender_Mixed',
    'Language',
    'English Requirement',
    'Dutch Requirement'
]

# Plotting variables
pp = pprint.PrettyPrinter(indent=4)
tqdm.tqdm.pandas(desc='progress-bar')
tqdm_auto.tqdm.pandas(desc='progress-bar')
# tqdm.notebook.tqdm().pandas(desc='progress-bar')
tqdm_auto.notebook_tqdm().pandas(desc='progress-bar')
# pbar = progressbar.ProgressBar(maxval=10)
mpl.style.use(f'{code_dir}/setup_module/apa.mplstyle-main/apa.mplstyle')
mpl.rcParams['text.usetex'] = False
font = {'family': 'arial', 'weight': 'normal', 'size': 10}
mpl.rc('font', **font)
plt.style.use('tableau-colorblind10')
plt.set_cmap('Blues')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.2f}'.format)


# Functions

In [ ]:
# Function to order categories
def categorize_df_results_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        try:
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')
        except ValueError as e:
            print(e)

    return df


### READ DATA

In [ ]:
with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)
assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_manual.shape}')


In [ ]:
df_manual = categorize_df_results_gender_age(df_manual)

### Set dataframes

In [ ]:
dataframes = {
    'df_manual': df_manual,
    # 'df_jobs': df_jobs,
}

# Analysis

### Analysis plan:
1. [Frequencies](#frequencies)
2. [Normality tests](#normality-tests) (normaltest, kurtosistest, shapiro, anderson, sphericity, and bartlett)

#### DF Manual:
1. One-way ANOVA, interactions, and post-hoc test
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
     - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
2. Logistic Regression  with all interaction (smf):
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
3. Logistic Specification Curve Analysis:
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)

#### DF Jobs:
1. One-way ANOVA, interactions, and post-hoc test
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
     - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
2. Logistic Regression with all interaction (smf):
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
3. Logistic Specification Curve Analysis:
   * **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
4. Multilevel OLS Regression with all interaction:
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
5. OLS Specification Curve Analysis:
   * **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
   * **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)


## Frequencies


In [ ]:
for df_name, df in dataframes.items():
    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')
    data_names = dvs+ivs_count+ivs_perc
    if df_name == 'df_jobs': data_names.extend(dvs_prob)
    print(rp.codebook(df[data_names]))

    for iv, dv in tqdm_product(ivs, dvs):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # FREQUENCIES
        print('\n')
        print('-' * 20)
        print('FREQUENCIES')
        print(f'IVs: {iv}')
        freq_iv=rp.summary_cat(df[[iv]]).round(3)
        print(freq_iv)
        freq_iv.to_csv(f'{table_save_path}frequencies {df_name} - {iv}.csv')
        print('-' * 20)
        print('\n')
        print('-' * 20)
        print(f'DVs: {dv}')
        freq_dv=rp.summary_cont(df[[dv]]).round(3)
        print(freq_dv)
        freq_dv.to_csv(f'{table_save_path}frequencies {df_name} - {dv}.csv')
        qq_plot = pg.qqplot(df[dv], dist='norm')
        plt.ion()
        plt.show()
        print('-' * 20)
        print('\n')


## Normality Tests


In [ ]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_jobs':
        dvs = dvs_prob

    for iv, dv in tqdm_product(ivs, dvs):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # Test of Normality
        norm = scipy.stats.normaltest(df[dv])

        print('='*80)
        print(f'{dv} Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Skewness-Kurtosis Test of Normality
        norm_sk = scipy.stats.kurtosistest(df[dv])

        print('='*80)
        print(f'{dv} Skewness-Kurtosis Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Shapir-Wilk Test of Normality
        norm_res = scipy.stats.shapiro(df[dv])

        print('='*80)
        print(f'{dv} Shapir-Wilk Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_res)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Anderson-Darling Test of Normality
        norm_and = scipy.stats.anderson(df[dv])

        print('='*80)
        print(f'{dv} Anderson-Darling Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # Significant
        print('\n')

        # NORMALITY TESTS
        print('NORMALITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        norm = pg.normality(data=df, dv=dv, group=iv).round(3)
        normal = bool(norm['normal'].to_string(index=False))
        print(f"{iv} x {dv} Normality test:\n{norm}")
        norm.to_csv(f"{table_save_path}normality {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # ANOVA SPHERICITY TEST
        print('SPHERICITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        spher_all = pg.sphericity(data=df, dv=dv, within=iv, method='mauchly')
        spher, test_stat, chisq, dof, pval = spher_all
        print('-' * 20)
        print(f"{iv} x {dv} Sphericity test:\n{spher} at p-value: {round(pval, 3)}, chi-square: {round(chisq, 3)}, degrees of freedom: {round(dof)}, Test statistic: {round(test_stat)}") # if p-value < 0.05, then the data are not spherically distributed = Multivariate analysis
        # spher.to_csv(f"{table_save_path}sphericity {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # BARTLETTS TESTS
        print("BARTLETT'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        bartlett = pg.homoscedasticity(data=df, dv=dv, group=iv, method='bartlett').round(3) #dv
        equal_var_bartlett = bool(bartlett['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Bartlett's test:\n{bartlett}")
        bartlett.to_csv(f"{table_save_path}bartlett's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # LEVENE'S TESTS
        print("LEVENE'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        levene = pg.homoscedasticity(data=df, dv=dv, group=iv, method='levene').round(3) #dv
        equal_var_levene = bool(levene['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Levene's test:\n{levene}")
        levene.to_csv(f"{table_save_path}levene's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')


In [ ]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_jobs':
        dvs = dvs_prob

    for iv, dv in tqdm_product(ivs, dvs):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # FREQUENCIES
        print('\n')
        print('-' * 20)
        print('FREQUENCIES')
        print(f'IVs: {iv}')
        freq_iv=rp.summary_cat(df[[iv]]).round(3)
        print(freq_iv)
        freq_iv.to_csv(f'{table_save_path}frequencies {df_name} - {iv}.csv')
        print('-' * 20)
        print('\n')
        print('-' * 20)
        print(f'DVs: {dv}')
        freq_dv=rp.summary_cont(df[[dv]]).round(3)
        print(freq_dv)
        freq_dv.to_csv(f'{table_save_path}frequencies {df_name} - {dv}.csv')
        qq_plot = pg.qqplot(df[dv], dist='norm')
        plt.ion()
        plt.show()
        print('-' * 20)
        print('\n')

        # Test of Normality
        norm = scipy.stats.normaltest(df[dv])

        print('='*80)
        print(f'{dv} Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Skewness-Kurtosis Test of Normality
        norm_sk = scipy.stats.kurtosistest(df[dv])

        print('='*80)
        print(f'{dv} Skewness-Kurtosis Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Shapir-Wilk Test of Normality
        norm_res = scipy.stats.shapiro(df[dv])

        print('='*80)
        print(f'{dv} Shapir-Wilk Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_res)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Anderson-Darling Test of Normality
        norm_and = scipy.stats.anderson(df[dv])

        print('='*80)
        print(f'{dv} Anderson-Darling Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # Significant
        print('\n')

        # NORMALITY TESTS
        print('NORMALITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        norm = pg.normality(data=df, dv=dv, group=iv).round(3)
        normal = bool(norm['normal'].to_string(index=False))
        print(f"{iv} x {dv} Normality test:\n{norm}")
        norm.to_csv(f"{table_save_path}normality {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # ANOVA SPHERICITY TEST
        print('SPHERICITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        spher_all = pg.sphericity(data=df, dv=dv, within=iv, method='mauchly')
        spher, test_stat, chisq, dof, pval = spher_all
        print('-' * 20)
        print(f"{iv} x {dv} Sphericity test:\n{spher} at p-value: {round(pval, 3)}, chi-square: {round(chisq, 3)}, degrees of freedom: {round(dof)}, Test statistic: {round(test_stat)}") # if p-value < 0.05, then the data are not spherically distributed = Multivariate analysis
        # spher.to_csv(f"{table_save_path}sphericity {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # BARTLETTS TESTS
        print("BARTLETT'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        bartlett = pg.homoscedasticity(data=df, dv=dv, group=iv, method='bartlett').round(3) #dv
        equal_var_bartlett = bool(bartlett['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Bartlett's test:\n{bartlett}")
        bartlett.to_csv(f"{table_save_path}bartlett's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # LEVENE'S TESTS
        print("LEVENE'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        levene = pg.homoscedasticity(data=df, dv=dv, group=iv, method='levene').round(3) #dv
        equal_var_levene = bool(levene['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Levene's test:\n{levene}")
        levene.to_csv(f"{table_save_path}levene's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')


## ANOVA

In [ ]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_jobs':
        dvs = dvs_prob

    for iv, dv in tqdm_product(ivs, dvs):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # LEVENE'S TESTS
        print("LEVENE'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        levene = pg.homoscedasticity(data=df, dv=dv, group=iv, method='levene').round(3) #dv
        equal_var_levene = bool(levene['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Levene's test:\n{levene}")
        levene.to_csv(f"{table_save_path}levene's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # SCIPY ANOVAS
        print('ANOVA SIGNIFICANCE')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        f_statistic, p_value = stats.f_oneway(
            df[dv][df[iv] == ivs_dict[iv][0]],
            df[dv][df[iv] == ivs_dict[iv][1]],
            df[dv][df[iv] == ivs_dict[iv][2]]
        )
        reject_H0 = p_value < alpha
        print('-' * 20)
        print(f'One-way ANOVA p-value: {p_value}. Rejected: {reject_H0}')
        print('~' * 20)

        # SUMMARY ANOVAS
        print('SUMMARY ANOVA')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        summary_aova = rp.summary_cont(df[dv].groupby(df[iv]), conf=0.95, decimals=3)
        print(summary_aova)
        summary_aova.to_csv(f'{table_save_path}summary anova {df_name} - {iv} x {dv}.csv')
        print('~' * 20)
        print('\n')

        # INTERACTION MODEL
        print(f'INTEACTION ANOVA {dv}')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        formula = f'{dv} ~ C({ivs[0]}, Treatment("{ivs_dict[ivs[0]][0]}")) + C({ivs[1]}, Treatment("{ivs_dict[ivs[1]][0]}")) + C({ivs[0]}, Treatment("{ivs_dict[ivs[0]][0]}")):C({ivs[1]}, Treatment("{ivs_dict[ivs[1]][0]}"))'
        model = ols(data = df, formula = formula).fit()
        anova_interaction_model = sm.stats.anova_lm(model, typ=2).round(3)
        print(model.summary())
        print(f"{iv} x {dv} ANOVA interaction model:\n{anova_interaction_model}")
        print('~' * 20)
        print('\n')

        if not equal_var_levene:
            # ONE-WAY ANOVA
            print('ONE-WAY ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova = pg.anova(data=df, dv=dv, between=iv, detailed=True).round(3)
            pg.print_table(anova)
            anova.to_csv(f'{table_save_path}one-way anova {df_name} - {iv} x {dv}.')
            print('~' * 20)
            print('\n')

            # TWO-WAY ANOVA
            print('ONE-WAY ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova = pg.anova(data=df, dv=dv, between=ivs, detailed=True).round(3)
            pg.print_table(anova)
            anova.to_csv(f'{table_save_path}two-way anova {df_name} - {ivs[0]} and {ivs[1]} x {dv}.csv')
            print('~' * 20)
            print('\n')

            # INTERACTION ANOVA
            print('INTERACTION ONE-WAY ANOVA')
            print('\n')
            print('~' * 20)
            print(f'Interaction: {ivs[0]} and {ivs[1]} x {dv}')
            anova_interaction = pg.anova(data=df, dv=dv, between=ivs, detailed=True).round(3)
            pg.print_table(anova_interaction)
            anova_interaction.to_csv(f'{table_save_path}interaction one-way anova {df_name} - {ivs} x {dv}.csv')
            print('~' * 20)
            print('\n')

            # TUKEY POST HOC
            print("POST HOC TUKEY'S ANOVA")
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova_pairwise_tukey = pg.pairwise_tukey(
                data=df, dv=dv, between=iv, effsize='eta-square'
            ).round(3)
            pg.print_table(anova_pairwise_tukey)
            anova_pairwise_tukey.to_csv(f'{table_save_path}post hoc tukey {df_name} - {iv} x {dv}.csv')
            print('~' * 20)
            print('\n')

        if equal_var_levene:
            # WELCH ANOVA
            print('WELCH ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova_welch = pg.welch_anova(data=df, dv=dv, between=iv).round(3)
            pg.print_table(anova_welch)
            anova_welch.to_csv(f'{table_save_path}welch anova {df_name} - {iv} x {dv}.csv')
            print('~' * 20)
            print('\n')

            ## INTERACTION ANOVA
            print('WELCH INTERACTION ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{ivs[0]} and {ivs[1]} x {dv}')
            anova_welch_interaction =  pg.welch_anova(data=df, dv=dv, between=ivs, detailed=True).round(3)
            pg.print_table(anova_welch_interaction)
            anova_welch_interaction.to_csv(f'{table_save_path}welch interaction anova {df_name} - {ivs} x {dv}.csv')
            print('~' * 20)
            print('\n')

            # KRUSKAL-WALLIS ANOVA
            print('KRUSKAL-WALLIS ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova_kruskal = pg.kruskal(data=df, dv=dv, between=iv).round(3)
            pg.print_table(anova_kruskal)
            anova_kruskal.to_csv(f'{table_save_path}kruskal-wallis anova {df_name} - {iv} x {dv}.csv')
            print('~' * 20)
            print('\n')

            ## INTERACTION ANOVA
            print('KRUSKAL-WALLIS INTERACTION ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{ivs[0]} and {ivs[1]} x {dv}')
            anova_kruskal_interaction = pg.kruskal(data=df, dv=dv, between=ivs, detailed=True).round(3)
            pg.print_table(anova_kruskal_interaction)
            anova_kruskal_interaction.to_csv(f'{table_save_path}kruskal-wallis interaction anova {df_name} - {ivs} x {dv}.csv')
            print('~' * 20)
            print('\n')

            # GAMES HOWELL POST HOC
            print('POST HOC GAMES HOWELL ANOVA')
            print('\n')
            print('~' * 20)
            print(f'{iv} x {dv}')
            anova_games_posthoc = pg.pairwise_gameshowell(
                data=df, dv=dv, between=iv, effsize='eta-square'
            ).round(3)
            pg.print_table(anova_games_posthoc)
            anova_games_posthoc.to_csv(f'{table_save_path}post hoc gameshowell {df_name} - {iv} x {dv}.csv')
            print('~' * 20)
            print('\n')
            print('+'*120)
            print('\n')


## Logistic Regression

In [ ]:
# Edit variable names for formula use: remove % and replace spaces with underscores
ivs_perc_ = list(map(lambda x: x.replace('%', 'percentage').replace(' ', '_'), ivs_perc))

# df_manual
df_manual_ = df_manual.copy()
df_manual_ = df_manual_.rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_manual_.columns})

# # df_jobs
# df_jobs_ = df_jobs.copy()
# df_jobs_ = df_jobs_.rename(columns={x: x.replace('%', 'percentage').replace(' ', '_') for x in df_jobs_.columns})


In [ ]:
ivs_perc_

In [ ]:
print([col for col in df_manual_.columns.to_list() if 'percentage' in col])

In [ ]:
dataframes_ = {
    'df_manual_': df_manual_,
    # 'df_jobs_': df_jobs_,
}


In [ ]:
for df_name, df in dataframes_.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_jobs':
        dvs = dvs_prob
    for dv in tqdm.tqdm(dvs):
        print('+'*120)
        print('\n')
        print(f'DEPENDENT VARIABLE: {dv}\n\nINDEPENDENT VARIABLE: {ivs_perc_}')
        print('\n')
        print('+'*120)

        # model = sm.Logit(endog=df[dv], exog=df[ivs_perc_], data=df)
        formula = f"{dv} ~ {ivs_perc_[0]} + {ivs_perc_[1]} + {ivs_perc_[2]} + {ivs_perc_[3]} + {ivs_perc_[0]}:{ivs_perc_[2]} + {ivs_perc_[0]}:{ivs_perc_[3]} + {ivs_perc_[1]}:{ivs_perc_[2]} + {ivs_perc_[1]}:{ivs_perc_[3]}"
        model = smf.logit(formula=formula, data=df)
        results = model.fit()
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))

        # display results
        print('~'*20)
        print('\n')
        print(f'SUMMARY RESULTS:\n{results.summary()}\n')
        print('-'*20)
        print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        print('~'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        print('-'*20)
        print(f'COEFFICIENT:\n{results.params}')
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        print(f'P-VALUES:\n{results.pvalues}')
        # print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        print(f'AIC:\n{results.aic:.2f}')
        print(f'BIC:\n{results.bic:.2f}')
        print(f'Coehn\'s F2:\n{results.prsquared:.3f}')
        print('-'*20)

        # save results
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))
        df_summary_results.to_csv(f'{table_save_path}logistic regression {df_name} - {dv} x Social Category Percentages.csv', index=False)


## Logistic Specification Curve Analysis

In [ ]:
# Variables
controls = [
        '% Sector per Workforce',
        'Job Description num_words',
        'English Requirement', 'Dutch Requirement',
        # 'Job Description num_unique_words',
        # 'Job Description num_chars',
        # 'Job Description num_chars_no_whitespact_and_punt',
        # 'Industry', 'Sector_n', 
        # 'Platform_LinkedIn', 'Platform_Indeed', 'Platform_Glassdoor',
]

# n_grams_counts = []
# for embedding_library, ngram_num in tqdm_product(embedding_libraries_list, ngrams_list):
#     controls.extend(
#         [
#             f'Job Description {embedding_library}_{ngram_num}grams_count',
#             f'Job Description {embedding_library}_{ngram_num}grams_abs_word_freq',
#             f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc',
#             f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc_cum'
#         ]
#     )

In [ ]:
sc.df_r.head()

In [ ]:
KeyError: "['x_exog', 'y_endog', 'conf_int', 'pvalues'] not in index"


In [ ]:
%%time
# Logistic Specification Curve Analysis for 0:1 Warmth and Competence
for df_name, df in dataframes.items():

    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')

    sc = specy.SpecificationCurve(df=df, y_endog=dvs, x_exog=ivs_perc, controls=controls)
    sc.fit(estimator=sm.Logit)
    df_results = sc.df_r

    # Plot and save
    for iv_perc, dv in tqdm_product(ivs_perc, dvs):
        print(f'{"="*5} RESULTS FOR {iv_perc.title()} ON {dv.title()} {"="*5}')

        for image_save_format in ['eps', 'png', 'svg']:
            save_path = f'{plot_save_path}Specification Curve - {iv_perc} x {dv}.{image_save_format}'
            sc.plot(preferred_spec=[iv_perc, dv], save_path=save_path)

    # Get statsmodels results and save
    ## Get gender only results
    gender_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Gender_Female_% per Sector', 'Gender_Male_% per Sector'] if item in x and len(x) == 2))
    df_results_gender = df_results[gender_mask]
    # Get age only results
    age_mask = df_results['Specification'].apply(lambda x: any(item for item in ['Age_Older_% per Sector', 'Age_Younger_% per Sector'] if item in x and len(x) == 2))
    df_results_age = df_results[age_mask]

    for df in [df_results_gender, df_results_age]:
        for idx, row in df.iterrows():
            print('\n')
            print('+'*20)
            print(f'{row["Specification"][0]} x {row["Specification"][1]}\n')
            print('+'*20)
            print(f'{row["Results"].summary()}')
            print('-'*20)

            # Save results to file
            df_to_save = pd.DataFrame(csv.reader(row["Results"].summary().as_csv().split('\n'), delimiter=','))
            df_to_save.to_csv(f'{table_save_path}specification curve - {row["Specification"][0]} x {row["Specification"][1]} {df_name}.csv', index=False)

    # Top 10 significant highest coefficients
    df_coeff_p = df_results.loc[sc.df_r['coeff_pvals'] < 0.05].sort_values(by=['Coefficient'], ascending=False)
    print(f"Top 10 significant highest coefficients:\n{df_coeff_p[['x_exog', 'y_endog', 'coeff_pvals', 'conf_int', 'pvalues']].head(10)}")

    print(f'{"="*5} END OF RESULTS FOR {iv_perc.title()} {"="*5}')


## OLS Regression

## OLS Specification Curve Analysis

In [ ]:
for df_name, df in dataframes.items():

    if df_name == 'df_jobs':
        dvs = [
            f'{dv}_Probability'
            for dv in dvs
        ]
    for dv, iv_dummy in tqdm_product(dvs, ivs_all_dummy):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv_dummy}')
        print('+'*120)

        model = sm.Logit(df[dv], df[ivs_all_dummy])
        results = model.fit()
        df_summary_results = pd.DataFrame(csv.reader(results.summary().as_csv().split('\n'), delimiter=','))

        # display results
        print(f'SUMMARY RESULTS:\n{results.summary()}')
        print(f'SUMMARY RESULTS2:\n{results.summary2()}')
        print(f'COEFFICIENT:\n{results.params}')
        print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        print(f'P-VALUES:\n{results.pvalues}')
        print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        print(f'AIC:\n{results.aic:.2f}'.format(results.aic))

        # display results
        print('~'*20)
        print(f'SUMMARY RESULTS:\n{results.summary()}')
        print('~'*20)
        # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
        print('-'*20)
        print(f'COEFFICIENT:\n{results.params}')
        # print(f'CONFIDENCE INTERVALS:\n{results.conf_int()}')
        print(f'P-VALUES:\n{results.pvalues}')
        print(f'ODDS RATIOS:\n{np.exp(results.params)}')
        print(f'AIC:\n{results.aic:.2f}'.format(results.aic))
        print(f'Coehn\'s F2: {results.rsquared_adj:.3f}')
        print('-'*20)

        # save results
        df_summary_results.to_csv(f'{table_save_path}logistic regression {df_name} - {dv} x {iv_dummy}.csv', index=False)


### OLS Regression

In [ ]:
# compute the vif for all given features
def compute_vif(df, considered_features):

    X = df[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1

    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif.loc[vif['Variable']!='intercept']
    return vif


In [ ]:
compute_vif(df_manual, ['Warmth', 'Competence']).sort_values('VIF', ascending=False)

In [ ]:
for dv in dvs:
    x = sm.add_constant(df['Gender_Female_% per Sector'])
    model = sm.OLS(df[dv], x)
    results = model.fit()
    print(results.summary())
    print(results.summary2())


In [ ]:
results.summary()

In [ ]:
df_manual.columns.tolist()

In [ ]:
ivs_all_dummy

In [ ]:
# df_manual[[f'{iv_dummy}_% per Sector' for iv_dummy in ivs_all_dummy if 'Mixed' not in iv_dummy]]

In [ ]:
for df_name, df in dataframes.items():

    if df_name == 'df_jobs':
        dvs = [
            f'{dv}_Probability'
            for dv in dvs
        ]
    for dv, iv_dummy in tqdm_product(dvs, ivs_all_dummy):
        if 'Mixed' not in iv_dummy:
            print('+'*120)
            print(f'Dependent Variable: {dv}\nIndependent Variable: {iv_dummy}')
            print('+'*120)

            x = sm.add_constant(df[[f'{iv_dummy}_% per Sector']])
            model = sm.OLS(df[dv], x)
            results = model.fit()

            # display results
            print('~'*20)
            print(f'SUMMARY RESULTS:\n{results.summary()}')
            print(results.summary())
            print('~'*20)
            # print(f'y = {results.params.const:.2f} + {results.params.x:.2f} * x')
            print('-'*20)
            print(f'AIC: {results.aic:.2f}'.format(results.aic))
            print('-'*20)
            print(f'Coehn\'s F2: {results.rsquared_adj:.3f}')
            print('-'*20)
            # table = sm.stats.anova_lm(results, typ=2)
            # print(table)

            # # Boxplot
            # boxplot = df.boxplot([dv], by = [ivs[0], ivs[1]],
            #                     figsize = (16, 9),
            #                     showmeans = True,
            #                     notch = True)

            # boxplot.set_xlabel('Categories')
            # boxplot.set_ylabel(dv)
            # # Creating a path to save the plot.
            # plt.ion()
            # plt.show()
            # plt.pause(.001)
            # boxplot.figure.savefig(
            #     f'{plot_save_path}Probability Boxplot - {ivs[0]} x {ivs[1]} x {dv}.{image_save_format}',
            #     format=image_save_format,
            #     dpi=3000,
            # )
            # plt.close()


### Multi-level Logistic Regression

In [ ]:
for df_name, df in dataframes.items():
    df['Intercept'] = 1

    print('+'*120)
    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')

    for dv in dvs:
        save_name = f'Multilevel Logistic Regression {df_name} - {list(iter(ivs_dict))[0]} + {list(iter(ivs_dict))[1]} x {dv}'
        endog = df[dv]
        exog0 = df[['Intercept', f'{list(iter(ivs_dict))[0]}']]
        exog1 = df[['Intercept', f'{list(iter(ivs_dict))[1]}']]

        print('='*80)
        print(f'Dependent Variable: {dv}:')
        print('='*80)
        # formula = f'dv ~ C(iv, Treatment("Reference_Category") + C(iv, Treatment("Reference_Category")'

        model = smf.mixedlm(
            formula=
            f'''{dv} ~ C({list(iter(ivs_dict))[0]}, Treatment("{ivs_dict[f'{list(iter(ivs_dict))[0]}'][0]}")) +
            C({list(iter(ivs_dict))[1]}, Treatment("{ivs_dict[f'{list(iter(ivs_dict))[1]}'][0]}"))''',
            data=df, groups=df['Job ID'].astype(str)
        ).fit()
        print(model.summary())
        df_model = pd.DataFrame(index=['Descriptives', 'Results'], columns=[f'{save_name}'])
        df_model[f'{save_name}']['Descriptives'] = model.summary().tables[0]
        df_model[f'{save_name}']['Results'] = model.summary().tables[1]

        df_model.to_csv(f'{table_save_path}{save_name.split(" model")[0].lower() + " model" + save_name.split(" model")[1]}.csv', header=True, index=True, index_label=['Index col: Descriptives and Results'])

        # Normality Tests (https://www.pythonfordatascience.org/mixed-effects-regression-python/)
        ## Residual and Kernal Density Estimate (KDE) Plot for Homoskedasticity
        fig = plt.figure(figsize = (16, 9))

        ax = sns.distplot(model.resid, hist = True, kde_kws = {"shade" : True, "lw": 1}, fit = scipy.stats.norm, kde=True, color='blue')

        ax.set_title(f"Kernal Density Estimate (KDE) Plot of Model Residuals (Blue) and Normal Distribution (Black)\n{save_name}")
        ax.set_xlabel("Residuals")
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # Q-Q Plot
        fig = plt.figure(figsize = (16, 9))
        ax = fig.add_subplot(111)

        qq = sm.qqplot(model.resid, dist = scipy.stats.norm, line = 's', ax = ax, color='blue', markerfacecolor='blue')
        ax.set_title(f"Q-Q Plot\n{save_name}",fontsize=15)
        ax.xaxis.get_label().set_fontsize(12)
        ax.yaxis.get_label().set_fontsize(12)
        ax.get_lines()[0].set_color('black')
        ax.get_lines()[0].set_linewidth('2')
        ax.get_lines()[1].set_color('black')
        ax.get_lines()[1].set_linewidth('2')
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # Test of Normality
        norm = scipy.stats.normaltest(model.resid)

        print('='*80)
        print(f'{dv} Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Skewness-Kurtosis Test of Normality
        norm_sk = scipy.stats.kurtosistest(model.resid)

        print('='*80)
        print(f'{dv} Skewness-Kurtosis Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Shapir-Wilk Test of Normality
        norm_res = scipy.stats.shapiro(model.resid)

        print('='*80)
        print(f'{dv} Shapir-Wilk Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_res)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Anderson-Darling Test of Normality
        norm_and = scipy.stats.anderson(model.resid)

        print('='*80)
        print(f'{dv} Anderson-Darling Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Residuals versus Fitted values (RVF) Plot for Homoskedasticity
        fig = plt.figure(figsize = (16, 9))

        ax = sns.scatterplot(y = model.resid, x = model.fittedvalues, color='blue')

        ax.set_title(f"Residuals versus Fitted values (RVF) Plot\n{save_name}")
        ax.set_xlabel("Fitted Values")
        ax.set_ylabel("Residuals")
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # White’s Lagrange Multiplier Test for Heteroscedasticity
        het_white_res = het_white(model.resid, model.model.exog)

        het_white_labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]

        print('='*80)
        print('White’s Lagrange Multiplier Test for Heteroscedasticity')
        print('-'*80)
        for key, val in dict(zip(het_white_labels, het_white_res)).items():
            print(key, val)
        print('\n')
        print('\n')
        print('+'*120)
        print('\n')


In [ ]:
for df_name, df in dataframes.items():
    df['Intercept'] = 1

    print('+'*120)
    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')

    for dv in dvs:
        save_name = f'Multilevel Logistic Regression {df_name} - {list(iter(ivs_dict))[0]} + {list(iter(ivs_dict))[1]} x {dv}'
        endog = df[dv]
        exog0 = df[['Intercept', f'{list(iter(ivs_dict))[0]}']]
        exog1 = df[['Intercept', f'{list(iter(ivs_dict))[1]}']]

        print('='*80)
        print(f'Dependent Variable: {dv}:')
        print('='*80)
        # formula = f'dv ~ C(iv, Treatment("Reference_Category") + C(iv, Treatment("Reference_Category")'

        model = smf.mixedlm(
            formula=
            f'''{dv} ~ C({list(iter(ivs_dict))[0]}, Treatment("{ivs_dict[f'{list(iter(ivs_dict))[0]}'][0]}")) +
            C({list(iter(ivs_dict))[1]}, Treatment("{ivs_dict[f'{list(iter(ivs_dict))[1]}'][0]}"))''',
            data=df, groups=df['Job ID'].astype(str)
        ).fit()
        print(model.summary())
        df_model = pd.DataFrame(index=['Descriptives', 'Results'], columns=[f'{save_name}'])
        df_model[f'{save_name}']['Descriptives'] = model.summary().tables[0]
        df_model[f'{save_name}']['Results'] = model.summary().tables[1]

        df_model.to_csv(f'{table_save_path}{save_name.split(" model")[0].lower() + " model" + save_name.split(" model")[1]}.csv', header=True, index=True, index_label=['Index col: Descriptives and Results'])

        # Normality Tests (https://www.pythonfordatascience.org/mixed-effects-regression-python/)
        ## Residual and Kernal Density Estimate (KDE) Plot for Homoskedasticity
        fig = plt.figure(figsize = (16, 9))

        ax = sns.distplot(model.resid, hist = True, kde_kws = {"shade" : True, "lw": 1}, fit = scipy.stats.norm, kde=True, color='blue')

        ax.set_title(f"Kernal Density Estimate (KDE) Plot of Model Residuals (Blue) and Normal Distribution (Black)\n{save_name}")
        ax.set_xlabel("Residuals")
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # Q-Q Plot
        fig = plt.figure(figsize = (16, 9))
        ax = fig.add_subplot(111)

        qq = sm.qqplot(model.resid, dist = scipy.stats.norm, line = 's', ax = ax, color='blue', markerfacecolor='blue')
        ax.set_title(f"Q-Q Plot\n{save_name}",fontsize=15)
        ax.xaxis.get_label().set_fontsize(12)
        ax.yaxis.get_label().set_fontsize(12)
        ax.get_lines()[0].set_color('black')
        ax.get_lines()[0].set_linewidth('2')
        ax.get_lines()[1].set_color('black')
        ax.get_lines()[1].set_linewidth('2')
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # Test of Normality
        norm = scipy.stats.normaltest(model.resid)

        print('='*80)
        print(f'{dv} Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Skewness-Kurtosis Test of Normality
        norm_sk = scipy.stats.kurtosistest(model.resid)

        print('='*80)
        print(f'{dv} Skewness-Kurtosis Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_sk)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Shapir-Wilk Test of Normality
        norm_res = scipy.stats.shapiro(model.resid)

        print('='*80)
        print(f'{dv} Shapir-Wilk Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_res)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Anderson-Darling Test of Normality
        norm_and = scipy.stats.anderson(model.resid)

        print('='*80)
        print(f'{dv} Anderson-Darling Test of Normality:')
        print('-'*80)
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # Significant
        print('\n')

        # Residuals versus Fitted values (RVF) Plot for Homoskedasticity
        fig = plt.figure(figsize = (16, 9))

        ax = sns.scatterplot(y = model.resid, x = model.fittedvalues, color='blue')

        ax.set_title(f"Residuals versus Fitted values (RVF) Plot\n{save_name}")
        ax.set_xlabel("Fitted Values")
        ax.set_ylabel("Residuals")
        plt.ion()
        fig.show('notebook')
        plt.pause(.001)

        # White’s Lagrange Multiplier Test for Heteroscedasticity
        het_white_res = het_white(model.resid, model.model.exog)

        het_white_labels = ["LM Statistic", "LM-Test p-value", "F-Statistic", "F-Test p-value"]

        print('='*80)
        print('White’s Lagrange Multiplier Test for Heteroscedasticity')
        print('-'*80)
        for key, val in dict(zip(het_white_labels, het_white_res)).items():
            print(key, val)
        print('\n')
        print('\n')
        print('+'*120)
        print('\n')


In [ ]:
[x for x in df_manual.columns.tolist() if 'Job Description' not in x]